You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os
#"""
# !rm k -r
if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Already up to date.
Processing /tf/k
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=56793 sha256=a8c6fae6b988fb91a61300472ec5130a94eae78d27dd80344df537a60f99f592
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wltyzx3/wheels/f0/08/19/56f64e8c8cc45b0390e5e7e2f634c4c1aa0212065044fb6442
Successfully built cai
  Attempting uninstall: cai
    Found existing installation: cai 0.1.4
    Uninstalling cai-0.1.4:
      Successfully uninstalled cai-0.1.4
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.6.0
Keras version: 2.6.0


In [ ]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [ ]:
# mixed_precision.set_global_policy('mixed_float16')

# Create folder structure with CIFAR-10 as png files.

In [ ]:
verbose=True
root_folder = 'cifar10-as-png'
data_dir = root_folder + '/train';
test_dir = root_folder + '/test';

if not os.path.isdir(root_folder):
  os.mkdir(root_folder)
  x_train, y_train, x_test, y_test = cai.datasets.load_dataset(tf.keras.datasets.cifar10, verbose=verbose, lab=False, bipolar=False)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_train)*255, y_train, dest_folder_name=data_dir)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_test)*255, y_test, dest_folder_name=test_dir)

In [ ]:
num_classes = 10
batch_size = 32
epochs = 50
target_size_x = 224
target_size_y = 224
seed = 12

In [ ]:
train_datagen = cai.util.create_image_generator(validation_split=0.1, rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.0, rescale=1./255)
test_datagen = cai.util.create_image_generator_no_augmentation(rescale=1./255)
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [ ]:
def work_on_keffnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 2, 13]:
      basefilename = 'JP30B27-keffnet-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.efficientnet.EfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes)
      else:
        model = cai.efficientnet.kEfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes,
          kType=kType)
        
      #opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model):
        model.summary(line_length=180)
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="training",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        validation_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="validation",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow_from_directory(
            directory=test_dir,
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_keffnet(show_model=True, run_fit=False, test_results=False)

Running: JP30B27-keffnet-CIFAR10--1
Model: "efficientnet-b0"
____________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                               Output Shape                            Param #              Connected to                                                
input_1 (InputLayer)                                       [(None, 224, 224, 3)]                   0                                                                                
____________________________________________________________________________________________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)                              (None, 225, 225, 3)                     0                    input_1[0][0]                                               
__________________________________

# Fitting

In [ ]:
work_on_keffnet(show_model=False, run_fit=True, test_results=True)

Running: JP30B27-keffnet-CIFAR10--1
Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
1407/1407 [==============================] - 362s 246ms/step - loss: 1.9325 - accuracy: 0.3196 - val_loss: 1.5513 - val_accuracy: 0.4456

Epoch 00001: val_accuracy improved from -inf to 0.44560, saving model to JP30B27-keffnet-CIFAR10--1-best_result.hdf5


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
1407/1407 [==============================] - 344s 243ms/step - loss: 1.4231 - accuracy: 0.4952 - val_loss: 1.4413 - val_accuracy: 0.5156

Epoch 00002: val_accuracy improved from 0.44560 to 0.51560, saving model to JP30B27-keffnet-CIFAR10--1-best_result.hdf5
Epoch 3/50
1407/1407 [==============================] - 344s 244ms/step - loss: 1.1906 - accuracy: 0.5872 - val_loss: 1.4636 - val_accuracy: 0.5466

Epoch 00003: val_accuracy improved from 0.51560 to 0.54660, saving model to JP30B27-keffnet-CIFAR10--1-best_result.hdf5
Epoch 4/50
1407/1407 [==============================] - 345s 244ms/step - loss: 1.0327 - accuracy: 0.6416 - val_loss: 1.0759 - val_accuracy: 0.6416

Epoch 00004: val_accuracy improved from 0.54660 to 0.64160, saving model to JP30B27-keffnet-CIFAR10--1-best_result.hdf5
Epoch 5/50
1407/1407 [==============================] - 343s 242ms/step - loss: 0.9387 - accuracy: 0.6750 - val_loss: 1.0716 - val_accuracy: 0.6536

Epoch 00005: val_accuracy improved from 0.64

# Test Results

In [ ]:
work_on_keffnet(show_model=False, run_fit=False, test_results=True)

Running: JP30B27-keffnet-CIFAR10--1
Found 10000 images belonging to 10 classes.
Best Model Results: JP30B27-keffnet-CIFAR10--1-best_result.hdf5
313/313 [==============================] - 16s 48ms/step - loss: 0.2061 - accuracy: 0.9333
loss 0.2060869187116623
acc 0.9333000183105469
Finished: JP30B27-keffnet-CIFAR10--1
Running: JP30B27-keffnet-CIFAR10-2
Found 10000 images belonging to 10 classes.
Best Model Results: JP30B27-keffnet-CIFAR10-2-best_result.hdf5
313/313 [==============================] - 39s 79ms/step - loss: 0.2408 - accuracy: 0.9224
loss 0.24077048897743225
acc 0.9223999977111816
Finished: JP30B27-keffnet-CIFAR10-2
Running: JP30B27-keffnet-CIFAR10-13
Found 10000 images belonging to 10 classes.
Best Model Results: JP30B27-keffnet-CIFAR10-13-best_result.hdf5
313/313 [==============================] - 38s 75ms/step - loss: 0.2028 - accuracy: 0.9375
loss 0.20281152427196503
acc 0.9375
Finished: JP30B27-keffnet-CIFAR10-13
